# Test Local Streaming Setup

In [ ]:
import asyncio

import aiohttp
import nest_asyncio

In [ ]:
nest_asyncio.apply()
sample_payload = {
    "payload": {
        "data": ["Apple", "Samsung", "OnePlus", "Google"]
    }
}
request_ep = "http://localhost:8080/python/produce"
response_ep = "http://localhost:8080/python/consume"

In [ ]:
# Async POST and GET polling using aiohttp

n = 1 # Number of times to send the request
poll_interval = 1 # Seconds between GET polls
max_polls = 10 # Maximum number of GET polls per request_id

async def poll_for_result(session: aiohttp.ClientSession, request_id: str) -> None:
    """Polls the response endpoint for the result corresponding to the given request_id."""
    for attempt in range(max_polls):
        async with session.get(f"{response_ep}/{request_id}") as resp:
            if resp.status == 200:
                data = await resp.json()
                if data.get("status") == "completed":
                    print(f"Result for request_id {request_id}: {data}")
                else:
                    print(f"Attempt {attempt + 1}: Result not ready yet for request_id {request_id}. Retrying in {poll_interval} seconds...")
            else:
                print(f"Attempt {attempt + 1}: Failed to fetch result for request_id {request_id}. Status code: {resp.status}")
        await asyncio.sleep(poll_interval)
    print(f"Max polls reached for request_id {request_id}. Moving to next request.")
    return None


async def post_and_poll() -> None:
    async with aiohttp.ClientSession() as session:
        tasks  = []
        for i in range(n):
            async with session.post(request_ep, json=sample_payload) as resp:
                if resp.status == 200:
                    data = await resp.json()
                    request_id = data.get("request_id")
                    print(f"Request {i + 1}: Sent payload. Received request_id: {request_id}")
                    if request_id:
                        tasks.append(poll_for_result(session, request_id))
                    else:
                        print(f"No request_id received. Status code: {resp.status}")
        # Running all polling tasks concurrently
        results = await asyncio.gather(*tasks)

In [ ]:
# Run the async function
asyncio.run(post_and_poll())